<a href="https://colab.research.google.com/github/bassamadnan/nlp-tf-fundamentals/blob/main/intro_to_nlp_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow
NLP has goal of deriving information out of natural language (could be sequences, text or speech.). Another common term for NLP problems is sequence-to-sequence problems (seq2seq)

In [16]:
# Check for GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-a47a4fc8-06c0-d96c-87c1-b4f7b43af1a6)


# Get helper functions


In [17]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-01-02 18:15:00--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-01-02 18:15:00 (56.3 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



## Get text dataset
The data set we will be using is Kaggle's introduction to NLP dataset, which can classify a tweet into a 'disaster' category, or not.
Source: https://www.kaggle.com/c/nlp-getting-started/data

In [18]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip the dataset
unzip_data("nlp_getting_started.zip")

--2023-01-02 18:15:00--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 172.217.218.128, 142.250.153.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2023-01-02 18:15:01 (79.9 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



## Visualizing text data set
Do note that collab allows about 12GB disk space, so it is reccomended to use datasets within the bound, using panads we may do it as:

In [19]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [20]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [21]:
# Clearly a disaster

In [22]:
# Shuffle training dataframe
train_df_shuffeled = train_df.sample(frac=1, random_state=42)
train_df_shuffeled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [23]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [24]:
# Data exploration
train_df["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

This dataset is roughly balanced fortunately, for classification of imbalanced data see: https://www.tensorflow.org/tutorials/structured_data/imbalanced_data

In [25]:
len(train_df), len(test_df)

(7613, 3263)

In [26]:
import random
random_index = random.randint(0, len(train_df)-5) # Create random indexes
for row in train_df_shuffeled[["text", "target"]][random_index:random_index+5].itertuples(): # Turns values in df to tuples
  _, text, target = row
  print(f"Target: {target} ", "Real disaster" if target > 0 else "(not a real disaster)")
  print(f"Test:\n{text}")
  print("---")

Target: 0  (not a real disaster)
Test:
G+: +https://t.co/dODXi41Y1CåÊis Body Bagging them lyrically! https://t.co/HlMyaAnrC9
---
Target: 0  (not a real disaster)
Test:
Our doctors and nurses in the new Pediatric Emergency Department are all specialized in child services! http://t.co/k1TMLWvjmJ
---
Target: 1  Real disaster
Test:
If you're in search of powerful content to improve your business or have been frustrated with the deluge of 'quantitÛ_https://t.co/64cyMG1lTG
---
Target: 1  Real disaster
Test:
Violent Forces Radio: Now Playing Acid Storm - Scourgue Of The Gods
TuneIn Player @ http://t.co/XsSgEdSbH4
---
Target: 0  (not a real disaster)
Test:
Ablaze for you Lord :D
---


# Split data into training and validation sets

In [28]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffeled["text"].to_numpy(),
                                                                            train_df_shuffeled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for reproducibility

In [29]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [30]:
train_sentences[:10], train_labels[:10] # First 10 tweets and labels 

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 